In [ ]:
!pip install wandb

In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F
import wandb
from torchsummary import summary

Застосування:
Медичні зображення: Bicubic добре підходить для задач, де важливі дрібні деталі, наприклад, аналіз родимок, рентгенівських знімків, МРТ тощо.
Графіка та фотографія: Використовується для масштабування фотографій, щоб уникнути пікселізації.

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200):
        super().__init__()
        self.out_channels = out_channels

        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),  # (600, 450)
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.Conv2d(out_channels, 2 * out_channels, kernel_size=3, stride=2, padding=1),  # (300, 225)
            nn.ReLU(),
            nn.Conv2d(2 * out_channels, 2 * out_channels, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.Conv2d(2 * out_channels, 4 * out_channels, kernel_size=3, stride=2, padding=1),  # (150, 113)
            nn.ReLU(),
            nn.Conv2d(4 * out_channels, 4 * out_channels, kernel_size=3, padding=1),  
            nn.ReLU(),
        )
        
        # Розраховуємо розмір після згорткових шарів: (4 * out_channels, 150, 113)
        self.flatten_size = 4 * out_channels * 150 * 113

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_size, latent_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = x.view(-1, 3, 600, 450)
        x = self.net(x)
        x = self.fc(x)
        return x


class Decoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200):
        super().__init__()
        self.out_channels = out_channels

        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 4 * out_channels * 150 * 113),
            nn.ReLU(),
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(4 * out_channels, 4 * out_channels, kernel_size=3, padding=1),  # (150, 113)
            nn.ReLU(),
            nn.ConvTranspose2d(4 * out_channels, 2 * out_channels, kernel_size=3, stride=2, padding=1, output_padding=0),  # (300, 225)
            nn.ReLU(),
            nn.ConvTranspose2d(2 * out_channels, 2 * out_channels, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.ConvTranspose2d(2 * out_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=0),  # (600, 450)
            nn.ReLU(),
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(out_channels, in_channels, kernel_size=3, padding=1),  
            nn.Sigmoid(),  # Для нормалізації виходу до [0, 1]
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 4 * self.out_channels, 150, 113)  # Відновлюємо форму для згорткових шарів
        x = self.conv(x)
        return x


class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        # self.encoder.to(device)
        self.decoder = decoder
        # self.decoder.to(device)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# Функція для обчислення коефіцієнта кореляції
def correlation_coefficient(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2))
    
    return numerator / denominator

# Функція для обчислення порогу
def calculate_threshold(correlation_coeffs):
    mu_c = np.mean(correlation_coeffs)
    sigma_c = np.std(correlation_coeffs)
    threshold = mu_c - 0.5 * sigma_c
    return threshold

In [ ]:
!wandb login 492953ddcda0576b6e6ebf89860aed0ccd177efe

In [ ]:
run_name = f"Autoencoder all examples"

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Skin cancer BASE model",

    # track hyperparameters and run metadata
    config={
    "epochs" : 10,
    "batch_size" : 16,
    "learning_rate" : 0.001,
    },
    name=run_name
)

In [ ]:
# Клас для завантаження зображень з папки
class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None, max_images=None):
        self.image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith('.jpg') or fname.endswith('.png')]
        if max_images is not None:
            self.image_paths = self.image_paths[:max_images]  # Обмежуємо кількість зображень до max_images
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
def train_autoencoder(model, dataloader, criterion, optimizer, scheduler, device, epochs):
    """
    Функція для тренування автоенкодера.
    
    Args:
        model: Модель автоенкодера.
        dataloader: Завантажувач даних.
        criterion: Функція втрат.
        optimizer: Оптимізатор.
        scheduler: Планувальник швидкості навчання.
        device: Пристрій (CPU або GPU).
        epochs: Кількість епох.

    Returns:
        Модель після тренування.
    """
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        
        for data in dataloader:
            data = data.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            
            # Враховуємо розмір для уникнення помилки розміру
            data_resized = F.interpolate(data, size=(600, 450), mode='bicubic', align_corners=False)
            
            loss = criterion(output, data_resized)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Середнє значення втрат для епохи
        average_loss = train_loss / len(dataloader)
        
        # Виведення результатів
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {average_loss}")
        
        # Логування втрат у wandb
        wandb.log({"epoch": epoch + 1, "loss": average_loss})
        
        # Оновлення швидкості навчання
        scheduler.step(average_loss)

    return model

In [ ]:
encoder = Encoder(in_channels=3, out_channels=16, latent_dim=1000)
decoder = Decoder(in_channels=3, out_channels=16, latent_dim=1000)
autoencoder_model = Autoencoder(encoder, decoder)

In [ ]:
summary(autoencoder_model, (3, 600, 450))

In [ ]:
summary(decoder, (200, )) 

In [ ]:
print(encoder)

In [ ]:
# Підготовка даних
image_folder = "/kaggle/input/ham1000-segmentation-and-classification/images"  # Шлях до папки з зображеннями
transform = transforms.Compose([
    transforms.Resize((600, 450), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor()
    # Якщо не потрібно нормалізувати, можете прибрати:
    # transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) 
])

dataset = CustomDataset(image_folder, transform)
dataloader = DataLoader(dataset, batch_size=wandb.config.batch_size, shuffle=True)

# # Створення та тренування автоенкодера
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = autoencoder.to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=wandb.config.learning_rate)
criterion = nn.MSELoss()  # Використовуємо BCE для бінарної класифікації
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4, factor=0.1)

# Виклик функції тренування
autoencoder = train_autoencoder(
    model=autoencoder,
    dataloader=dataloader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=wandb.config.epochs
)

# Збереження моделі
torch.save(autoencoder.state_dict(), "cnn_autoencoder.pth")
wandb.save("cnn_autoencoder.pth")

In [ ]:
# Отримання вихідного зображення після реконструкції
test_image = Image.open("/kaggle/input/dddddsd/photo_5_2025-01-05_14-16-42.jpg").convert('RGB')
test_image = transform(test_image).unsqueeze(0).to(device)
output_image = autoencoder(test_image).detach().cpu().numpy()

# Обчислення коефіцієнту кореляції
test_image_resized = transforms.functional.resize(test_image, size=(600, 456), interpolation=transforms.InterpolationMode.BICUBIC)
input_pixels = test_image_resized.squeeze(0).cpu().numpy().flatten()

reconstructed_pixels = output_image.flatten()
corr_value = correlation_coefficient(input_pixels, reconstructed_pixels)
print(f"Correlation Coefficient: {corr_value}")

# Масив кореляцій для класу (приклад)
correlation_coeffs = np.random.rand(100)  # Масив кореляцій для класу

# Обчислення порогу для цього класу
threshold = calculate_threshold(correlation_coeffs)
print(f"Threshold: {threshold}")

# Перевірка чи зображення належить тому ж класу
if corr_value > threshold:
    print("Зображення належить тому ж класу.")
else:
    print("Зображення не належить тому ж класу.")